# Meta-analysis of GWAS traits
- for each signal at each locus, perform meta-analysis to get one single GWAS signal (which is equal to the combined signal of all colocalizing traits)
- colocalize this meta-signal with eQTLs (only for eGenes that already colocalize with one of the individual GWAS signals, PPH4 > 0.8)


In [1]:
setwd("/frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc")

source("script/functions.R"  )


In [2]:
dir.create("pipeline/5.5.meta_analysis"           , showWarnings = FALSE)
dir.create("pipeline/5.5.meta_analysis/coloc_data", showWarnings = FALSE)


# Get GWAS data

In [3]:
suppressPackageStartupMessages(library(kohonen))


In [4]:
manifest   = add_rownames(fread("pipeline/1.1.sumstats/manifest.txt"                     , sep = "\t", header = TRUE, data.table = FALSE))
loci       = add_rownames(fread("pipeline/1.2.genomewide_significant_loci/loci.txt"      , sep = "\t", header = TRUE, data.table = FALSE))
loci2study = add_rownames(fread("pipeline/1.2.genomewide_significant_loci/loci2study.txt", sep = "\t", header = TRUE, data.table = FALSE))


In [5]:
populations = c('meta','AFR','AMR','CSA','EAS','EUR','MID')

In [6]:
moloc_df       = fread  ("pipeline/2.2.moloc/moloc.txt", sep = "\t", header = TRUE, data.table = FALSE)
moloc_list     = readRDS("pipeline/2.2.moloc/moloc_list.rds")
moloc_map_list = readRDS("pipeline/2.2.moloc/moloc_map_list.rds")


In [7]:
moloc_loci_groups = moloc_map_list[["som"]][["input"]]
moloc_loci2class  = moloc_map_list[["loci2class"]]
moloc_trait2class = moloc_map_list[["trait2class"]]

moloc_loci2class$class = paste0("V", moloc_loci2class$som)

rownames(moloc_loci2class) = moloc_loci2class$id

## Get eQTL coloc data

In [8]:
qtl_list = readRDS("/frazer01/projects/CARDIPS/analysis/cardiac_qtls_combined/input/qtl/qtls.RDS"   )
exp_list = readRDS("/frazer01/projects/CARDIPS/analysis/cardiac_qtls_combined/input/expdata_qtl.rds")

In [9]:
phenotypes  = qtl_list$phenotypes
qtls        = qtl_list$qtl
coordinates = exp_list$coordinates
rownames(coordinates) = coordinates$transcript_id

In [10]:
coloc_qtls = fread("pipeline/4.1.coloc_qtls/coloc_eqtls.txt", sep = "\t", header = TRUE, data.table = FALSE)


# Data for meta analysis
- https://bookdown.org/MathiasHarrer/Doing_Meta_Analysis_in_R/fixed.html

In [11]:
suppressPackageStartupMessages(library(meta))
suppressPackageStartupMessages(library(coloc))


In [12]:
coloc_list        = readRDS("pipeline/5.4.analyze_coloc_qtl_som_maps/coloc_list.rds")
locus2coloc       = fread  ("pipeline/5.4.analyze_coloc_qtl_som_maps//locus2coloc.txt", sep = "\t", header = TRUE, data.table = FALSE)
intersected       = fread  ("pipeline/4.1.coloc_qtls/intersected_qtls_loci.txt"       , sep = "\t", header = TRUE, data.table = FALSE)
qtls2test         = fread  ("pipeline/4.1.coloc_qtls/qtls2test.txt"                   , sep = "\t", header = TRUE, data.table = FALSE)
intersected       = intersected[ intersected$phenotype %in% c("rna", "isoform"),]


# Create qsub for each locus

In [13]:
run_qsub = function(id)
{
    qsub = paste("qsub", paste(getwd(), "script", "5.5.coloc_qtl_meta_analysis.sh", sep = "/"), id)
    
    system(qsub)
    
    #message(qsub)
}

invisible(lapply(locus2coloc$id, run_qsub))


## Check what went wrong

In [14]:
locus2file        = locus2coloc
locus2file$file   = paste0("pipeline/5.5.meta_analysis/coloc_data/", locus2file$id, ".rds")
locus2file$exists = file.exists(locus2file$file)

locus2file[ locus2file$exists == FALSE, ]

som,id,locus,cluster,class,traits,traits_n,eqtls,eqtls_n,file,exists
<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<lgl>


# SCRATCH - Meta-analysis of GWAS traits
- https://bookdown.org/MathiasHarrer/Doing_Meta_Analysis_in_R/fixed.html

In [14]:
suppressPackageStartupMessages(library(meta))
suppressPackageStartupMessages(library(coloc))


In [15]:
coloc_list        = readRDS("pipeline/5.4.analyze_coloc_qtl_som_maps/coloc_list.rds")
locus2coloc       = fread  ("pipeline/5.4.analyze_coloc_qtl_som_maps//locus2coloc.txt", sep = "\t", header = TRUE, data.table = FALSE)
intersected       = fread  ("pipeline/4.1.coloc_qtls/intersected_qtls_loci.txt"       , sep = "\t", header = TRUE, data.table = FALSE)
qtls2test         = fread  ("pipeline/4.1.coloc_qtls/qtls2test.txt"                   , sep = "\t", header = TRUE, data.table = FALSE)
intersected       = intersected[ intersected$phenotype %in% c("rna", "isoform"),]


In [60]:
get_gwas_data = function(coord, gwas_file)
{
    my_head              = colnames(fread(cmd = paste("zcat", gwas_file, "|", "head -n 2"), sep = "\t", header = TRUE, data.table = FALSE))
    gwas_data            = suppressWarnings(tabix.read.table(gwas_file, coord, col.names = TRUE, stringsAsFactors = FALSE))
    colnames(gwas_data)  = my_head
    gwas_data$id         = paste("VAR", gwas_data$chr, gwas_data$pos, gwas_data$ref, gwas_data$alt, sep = "_")
    rownames(gwas_data)  = gwas_data$id
    
    if("af_meta"          %in% colnames(gwas_data)){gwas_data$af = gwas_data$af_meta         }
    if("af_controls_meta" %in% colnames(gwas_data)){gwas_data$af = gwas_data$af_controls_meta}
    
    return(gwas_data)
}

get_qtl_data = function(qtl_file)
{
    indata = fread(qtl_file, sep = "\t", header = TRUE, data.table = FALSE)
    return(indata)
}

calculate_meta = function(id, traits_cluster, tometa_te, tometa_se, tometa_pval, tometa_af)
{
    totest = data.frame(trait = traits_cluster, 
                        TE    = as.numeric(tometa_te  [id, traits_cluster]),
                        seTE  = as.numeric(tometa_se  [id, traits_cluster]),
                        pval  = as.numeric(tometa_pval[id, traits_cluster])
                       )
    totest = totest[is.na(totest$TE) == FALSE,]
    
    if(nrow(totest) > 1)
    {
        m = metagen(TE          = TE,
                    seTE        = seTE,
                    data        = totest,
                    studlab     = paste(trait),
                    comb.fixed  = TRUE,
                    comb.random = FALSE,
                    prediction  = TRUE,
                    sm="SMD")
        
        out = data.frame(id       = id,
                         beta     = m$TE.fixed  ,  
                         se       = m$seTE.fixed,  
                         pval     = m$pval.fixed,
                         af       = mean(as.numeric(tometa_af[id,]), na.rm = TRUE),
                         pval_het = m$pval.Q
                        )
    }else
    {
        out = data.frame(id       = id,
                         beta     = totest$TE  ,  
                         se       = totest$seTE,  
                         pval     = totest$pval,
                         af       = mean(as.numeric(tometa_af[id,]), na.rm = TRUE),
                         pval_het = 1
                        )
    }
    
    if(nrow(out[is.na(out$pval_het) == TRUE, ]) > 0){out[is.na(out$pval_het) == TRUE, "pval_het"] = 1}
    
    return(out)
}

create_dataset = function(study, trait_type, totest, variants, populations, manifest)
{
    if(trait_type %in% c("categorical", "icd10", "phecode"))
    {
        n = sum(manifest[study, paste("n_cases", populations, sep = "_")]) + sum(manifest[study, paste("n_controls", populations, sep = "_")])
    }
    if(trait_type %in% c("biomarkers", "continuous"))
    {
        n = sum(manifest[study, paste("n_cases", populations, sep = "_")])
    }
    if(trait_type == "qtl")
    {
        n = 966
    }

    rownames(totest) = totest$id
    dataset = list(snp     = variants, 
                   pvalues = totest     [variants, "pval"], 
                   N       = n, 
                   MAF     = totest     [variants, "af"], 
                   type    = "quant")
    
    return(dataset)
}

finemap_gwas = function(dataset)
{
    if(length(dataset$pvalues) > 10000)
    {
        new_dataset = data.frame(pval = dataset$pvalues, maf = dataset$MAF, snp = dataset$snp)
        new_dataset = new_dataset[order(new_dataset$pval),]
        new_dataset = new_dataset[1:10000,]
        dataset     = list(pvalues = new_dataset$pval, MAF = new_dataset$maf, snp = new_dataset$snp, N = dataset$N, sdY = 1, type = dataset$type)
    }
    
    finemap           = finemap.abf(dataset = dataset)
    finemap           = finemap[,c("snp", "pvalues.", "SNP.PP")]
    colnames(finemap) = c("id", "pval", "pp_snp")
    myres             = finemap
    myres             = myres[order(myres$pp_snp, decreasing = TRUE), ]
    myres$cum         = cumsum(myres$pp_snp)
    myres$cs          = FALSE
    to_cs             = myres[myres$cum >= 0.99, ][1, "id"]
    
    myres[myres$cum < 0.99 | myres$id == to_cs, "cs"] = TRUE

    credible_set = myres[ myres$cs == TRUE, "id"]
    
    return(list(pp = myres, credible_set = credible_set))
}

quiet = function(x) 
{ 
    sink(tempfile()) 
    on.exit(sink()) 
    invisible(force(x)) 
} 

run_coloc = function(id, study1, study2, populations, totest1, totest2, qtls2test, manifest)
{
    trait_type1 = manifest[study1, "trait_type"]
    trait_type2 = "qtl"
    variants1   = totest1[is.na(totest1$pval) == FALSE & is.na(totest1$af) == FALSE, "id"]
    types       = sort  (qtls2test[qtls2test$transcript_id == study2, "type"     ])
    phenotype   = unique(qtls2test[qtls2test$transcript_id == study2, "phenotype"])
    
    out = lapply(types, function(type)
    {
        totest_type = totest2[totest2$type == type,]
        variants2   = totest_type$id
        variants    = intersect(variants1, variants2)

        if(length(variants) > 100)
        {
            dataset1        = create_dataset(study1, trait_type1, totest1    , variants, populations, manifest)
            dataset2        = create_dataset(study2, trait_type2, totest_type, variants, c()        , manifest)
            coloc_mapped    = coloc.abf(dataset1 = dataset1, dataset2 = dataset2) 
            probs           = as.data.frame(t(coloc_mapped$summary))
            myres           = coloc_mapped$results
            myres           = myres[, c(which(colnames(myres) == "snp"), ncol(myres))]
            colnames(myres) = c("id", "pp_snp")
            myres           = myres[order(myres$pp_snp, decreasing = TRUE), ]
            out             = cbind(probs, myres[1, ])
            
            out = list(by_snp = myres, pp = out)
        }else
        {
            out = list(by_snp = data.frame(id = "", pp_snp = 0),
                       pp     = data.frame(nsnps = 0, PP.H0.abf = 1, PP.H1.abf = 0, PP.H2.abf = 0, PP.H3.abf = 0, PP.H4.abf = 0, id = "", pp_snp = 0)
                      )
        }

        return(out)
    })
    
    names(out) = paste("type", types)
    return(out)
}


meta_analysis = function(id, moloc_loci_groups, coloc_list, loci, manifest, intersected, qtls2test, to_return = FALSE)
{
    som            = coloc_list[[id]][["som"           ]]
    eqtls          = coloc_list[[id]][["eqtls"         ]]
    traits         = coloc_list[[id]][["traits"        ]]
    coloc          = coloc_list[[id]][["coloc"         ]]   
    traits_cluster = data.frame(trait = colnames(moloc_loci_groups), x = as.numeric(moloc_loci_groups[id,]))
    traits_cluster = traits_cluster[ traits_cluster$x == 1, "trait"]
    
    locus         = som[1, "locus"]
    coloc         = coloc[ coloc$PP.H4.abf >= 0.8,]
    coloc         = coloc[order(coloc$PP.H4.abf, decreasing = TRUE),]
    coloc$tr2type = paste(coloc$transcript_id, coloc$type)
    coord         = loci[locus,]
    coord         = paste0(coord$chrom, ":", coord$from, "-", coord$to)
    
    # get GWAS data
    totest_gwas_list        = lapply(traits_cluster, function(gwas){get_gwas_data(coord, manifest[gwas, "sumstat_file"])})
    names(totest_gwas_list) = traits_cluster
    
    if(length(totest_gwas_list) > 1)
    {
        gwas = totest_gwas_list[[1]]
        gwas = data.frame(id       = gwas$id,
                          beta     = gwas$beta_meta,
                          se       = gwas$se_meta  ,
                          pval     = gwas$pval_meta,
                          af       = gwas$af,
                          pval_het = 1
                         )

        var_ids     = sort(unique(unlist(lapply(totest_gwas_list, function(x){x$id}))))
        tometa      = as.data.frame(matrix(0, nrow = length(var_ids), ncol = length(totest_gwas_list), dimnames = list(var_ids, traits_cluster)))
        tometa_te   = tometa
        tometa_se   = tometa
        tometa_pval = tometa
        tometa_af   = tometa
        
        for(trait in traits_cluster)
        {
            x = totest_gwas_list[[trait]]
            tometa_te  [ x$id, trait] = x$beta_meta
            tometa_se  [ x$id, trait] = x$se_meta
            tometa_pval[ x$id, trait] = x$pval_meta
            tometa_af  [ x$id, trait] = x$af
        }
        
        gwas = suppressWarnings(as.data.frame(rbindlist(lapply(var_ids, function(id){calculate_meta(id, traits_cluster, tometa_te, tometa_se, tometa_pval, tometa_af)}))))
    }else
    {
        gwas = totest_gwas_list[[1]]
        gwas = data.frame(id       = gwas$id,
                          beta     = gwas$beta_meta,
                          se       = gwas$se_meta  ,
                          pval     = gwas$pval_meta,
                          af       = gwas$af,
                          pval_het = 1
                         )
    }
    
    rownames(gwas)    = gwas$id
    trait             = traits_cluster[[1]]
    tocoloc           = intersected[ intersected$locus == locus, ]
    trait_type1       = manifest[trait, "trait_type"]
    trait_type2       = "qtl"
    populations       = unlist(strsplit(manifest[trait, "pops"], ","))
    totest_qtl        = lapply(tocoloc$qtl_file, get_qtl_data)
    names(totest_qtl) = tocoloc$transcript_id
    qtls2test         = qtls2test[ qtls2test$transcript_id %in% tocoloc$transcript_id,]
    dataset           = create_dataset(trait, 
                                       manifest[trait, "trait_type"], 
                                       gwas, 
                                       gwas[is.na(gwas$pval) == FALSE & is.na(gwas$af) == FALSE, "id"], 
                                       populations, 
                                       manifest)
    
    finemapped = finemap_gwas(dataset)

    #transcript_id = names(totest_qtl)[[1]]
    #out           = run_coloc(id, trait, transcript_id, populations, gwas, totest_qtl[[transcript_id]], qtls2test, manifest)
    
    coloc_list = quiet(lapply(names(totest_qtl), function(transcript_id)
    {
        run_coloc(id, trait, transcript_id, populations, gwas, totest_qtl[[transcript_id]], qtls2test, manifest)
    }))
    
    names(coloc_list) = names(totest_qtl)
    
    out = list(som                 = som,
               traits_cluster      = traits_cluster   ,
               eqtls               = qtls2test        ,
               eqtl_transcript_ids = names(totest_qtl),
               eqtl_data           = totest_qtl       ,
               gwas_data           = totest_gwas_list ,
               meta_analysis_gwas  = gwas             ,
               finemapped_gwas     = finemapped       ,
               coloc               = coloc_list
              )
    
    saveRDS(out, paste0("pipeline/5.5.meta_analysis/coloc_data/", id, ".rds"))
    
    if(to_return == TRUE){return(out)}
}

id = "5_136864391_137972681.2" # atrial fibrillation + pulse rate
id = "7_19692053_19892053.1" 

x = meta_analysis(id, moloc_loci_groups, coloc_list, loci, manifest, intersected, qtls2test, TRUE)


In [61]:
str(x)

List of 9
 $ som                :'data.frame':	1 obs. of  5 variables:
  ..$ som    : num 48
  ..$ id     : chr "7_19692053_19892053.1"
  ..$ locus  : chr "7_19692053_19892053"
  ..$ cluster: int 14
  ..$ class  : chr "V48"
 $ traits_cluster     : chr "categorical-20003-both_sexes-1140866738"
 $ eqtls              :'data.frame':	1 obs. of  8 variables:
  ..$ gene_id      : chr "ENSG00000105849.6_5"
  ..$ gene_name    : chr "TWISTNB"
  ..$ transcript_id: chr "ENSG00000105849.6_5"
  ..$ type         : int 0
  ..$ chrom        : int 7
  ..$ start        : int 19735084
  ..$ end          : int 19748660
  ..$ phenotype    : chr "rna"
 $ eqtl_transcript_ids: chr "ENSG00000105849.6_5"
 $ eqtl_data          :List of 1
  ..$ ENSG00000105849.6_5:'data.frame':	5216 obs. of  13 variables:
  .. ..$ chrom     : int [1:5216] 7 7 7 7 7 7 7 7 7 7 ...
  .. ..$ pos       : int [1:5216] 19240546 19241659 19241735 19241953 19241980 19247180 19247377 19248160 19249169 19249778 ...
  .. ..$ ref       : chr [

In [59]:
x$finemapped_gwas

id,pval,pp_snp
<chr>,<dbl>,<dbl>
VAR_7_19692124_C_T,0.6139000,4.811418e-08
VAR_7_19692174_C_T,0.7018000,4.558298e-08
VAR_7_19692670_C_G,0.9668000,1.481438e-07
VAR_7_19692863_C_G,0.5683000,4.984796e-08
VAR_7_19693464_T_C,0.5545000,1.909496e-07
VAR_7_19693604_A_G,0.4721000,2.106482e-07
VAR_7_19693626_C_T,0.0007141,3.779624e-05
VAR_7_19693755_C_T,0.0009416,2.936939e-05
VAR_7_19693996_G_A,0.0006517,4.135361e-05


In [29]:
finemap_gwas = function(dataset)
{
    if(length(dataset$pvalues) > 10000)
    {
        new_dataset = data.frame(pval = dataset$pvalues, maf = dataset$MAF, snp = dataset$snp)
        new_dataset = new_dataset[order(new_dataset$pval),]
        new_dataset = new_dataset[1:10000,]
        dataset     = list(pvalues = new_dataset$pval, MAF = new_dataset$maf, snp = new_dataset$snp, N = dataset$N, sdY = 1, type = dataset$type)
    }
    
    finemap           = finemap.abf(dataset = dataset)
    finemap           = finemap[,c("snp", "pvalues.", "SNP.PP")]
    colnames(finemap) = c("id", "pval", "pp_snp")
    
    return(finemap)
    
    myres             = finemap
    myres             = myres[order(myres$pp_snp, decreasing = TRUE), ]
    myres$cum         = cumsum(myres$pp_snp)
    myres$cs          = FALSE
    to_cs             = myres[myres$cum >= 0.99, ][1, "id"]
    
    myres[myres$cum < 0.99 | myres$id == to_cs, "cs"] = TRUE

    credible_set = myres[ myres$cs == TRUE, "id"]
    
    return(list(pp = myres, credible_set = credible_set))
}

dataset = x[[1]]
trait_type = x[[2]]

finemap_gwas(dataset)




id,pval,pp_snp
<chr>,<dbl>,<dbl>
VAR_19_45242740_A_G,1.057829e-264,9.987847e-01
VAR_19_45418790_T_C,9.430279e-262,1.215255e-03
VAR_19_45416178_G_T,6.215120e-246,1.916881e-19
VAR_19_45416741_C_T,7.463390e-244,1.604392e-21
VAR_19_45220896_T_C,4.514966e-220,3.379888e-45
VAR_19_45384338_G_A,4.761959e-219,2.395260e-46
VAR_19_45687093_C_T,3.200399e-206,2.782476e-59
VAR_19_45384105_C_T,5.042943e-206,2.347705e-59
VAR_19_45372959_C_T,2.187813e-202,6.078236e-63


In [ ]:
finemap = x[[2]]
myres   = finemap
myres           = myres[order(myres$pp_snp, decreasing = TRUE), ]
myres$cum       = cumsum(myres$pp_snp)
myres$cs        = FALSE
to_cs           = myres[myres$cum >= 0.99, ][1, "id"]

myres[myres$cum < 0.99 | myres$id == to_cs, "cs"] = TRUE

credible_set = myres[ myres$cs == TRUE, "id"]

str(myres)

myres

# Integrate coloc with moloc data

In [ ]:
get_coloc_qtl_by_locus = function(id, moloc_loci2class, coloc_qtls)
{
    locus       = moloc_loci2class[id, "locus"  ]
    som         = moloc_loci2class[id, "class"  ]
    cluster     = moloc_loci2class[id, "cluster"]
    traits2test = data.frame(trait = colnames(moloc_trait2class), weight = as.numeric(moloc_trait2class[som,]))
    traits      = traits2test[traits2test$weight >= 0.5, "trait"]
    coloc       = coloc_qtls[ coloc_qtls$locus == locus & coloc_qtls$pop == "meta" & coloc_qtls$gwas %in% traits,]
    
    out = list(som            = moloc_loci2class[id,],
               traits_cluster = traits,
               eqtls          = coloc[ coloc$PP.H4.abf >= 0.8, "transcript_id"],
               traits         = traits2test,
               coloc          = coloc
              )
    
    #return(coloc[ coloc$PP.H4.abf >= 0.8,])
    return(out)
}

id = "5_130403286_131933599.5"
id = "5_130403286_131933599.3"

coloc_list        = lapply(moloc_loci2class$id, function(id){get_coloc_qtl_by_locus(id, moloc_loci2class, coloc_qtls)})
names(coloc_list) = moloc_loci2class$id

saveRDS(coloc_list, "pipeline/5.4.analyze_coloc_qtl_som_maps/coloc_list.rds")

In [ ]:
locus2coloc = as.data.frame(rbindlist(lapply(coloc_list, function(x)
{
    out          = x$som
    out$traits   = paste(sort(unique(x$traits_cluster)), collapse = "; ")
    out$traits_n = length(unique(x$traits_cluster))
    out$eqtls    = paste(sort(unique(x$eqtls)), collapse = "; ")
    out$eqtls_n  = length(unique(x$eqtls))
    
    return(out)
})), stringsAsFactors = FALSE)

fwrite(locus2coloc, "pipeline/5.4.analyze_coloc_qtl_som_maps/locus2coloc.txt", sep = "\t", col.names = TRUE, row.names = FALSE)

In [ ]:
locus2coloc[ locus2coloc$id == "17_40144007_48070076.2",]

In [ ]:
totest = locus2coloc[ locus2coloc$traits_n > 0 & rowSums(locus2coloc[,c("traits_n", "eqtls_n")]) > 1, "id"]

length(totest)

# Analyze loci

In [ ]:
suppressPackageStartupMessages(library(coloc))

In [ ]:
get_gwas_data = function(coord, gwas_file)
{
    my_head              = colnames(fread(cmd = paste("zcat", gwas_file, "|", "head -n 2"), sep = "\t", header = TRUE, data.table = FALSE))
    gwas_data            = suppressWarnings(tabix.read.table(gwas_file, coord, col.names = TRUE, stringsAsFactors = FALSE))
    colnames(gwas_data)  = my_head
    gwas_data$id         = paste("VAR", gwas_data$chr, gwas_data$pos, gwas_data$ref, gwas_data$alt, sep = "_")
    rownames(gwas_data)  = gwas_data$id
    
    return(gwas_data)
}

create_dataset = function(study, trait_type, totest, variants, pop, manifest)
{
    #message(paste(study, trait_type, pop))
    if(trait_type %in% c("categorical", "icd10", "phecode"))
    {
        populations = unlist(strsplit(manifest[study, "pops"], ","))
        if(pop == "meta"){n = sum(manifest[study, paste("n_cases", populations, sep = "_")]) + sum(manifest[study, paste("n_controls", populations, sep = "_")])}
        if(pop != "meta"){n =     manifest[study, paste("n_cases", pop        , sep = "_")]  +     manifest[study, paste("n_controls", pop        , sep = "_")] }
        
        if(pop == "meta"){s = sum(manifest[study, paste("n_cases", populations, sep = "_")]) / n}
        if(pop != "meta"){s =     manifest[study, paste("n_cases", pop        , sep = "_")]  / n}
        
        totest  = totest[is.na(totest[,paste("pval", pop, sep = "_")]) == FALSE & is.na(totest[,paste("af_controls", pop, sep = "_")]) == FALSE, ]
        dataset = list(snp = variants, pvalues = totest[variants, paste("pval", pop, sep = "_")], N = n, s = s, MAF = totest[variants, paste("af_controls", pop, sep = "_")], type = "cc")
        
    }
    if(trait_type %in% c("biomarkers", "continuous"))
    {
        populations = unlist(strsplit(manifest[study, "pops"], ","))
        if(pop == "meta"){n = sum(manifest[study, paste("n_cases", populations, sep = "_")])}
        if(pop != "meta"){n =     manifest[study, paste("n_cases", pop        , sep = "_")] }
        
        totest  = totest[is.na(totest[,paste("pval", pop, sep = "_")]) == FALSE & is.na(totest[,paste("af", pop, sep = "_")]) == FALSE, ]
        dataset = list(snp = variants, pvalues = totest[variants, paste("pval", pop, sep = "_")], N = n, MAF = totest[variants, paste("af", pop, sep = "_")], type = "quant")
    }
    if(trait_type == "qtl")
    {
        rownames(totest) = totest$id
        dataset = list(snp     = variants, 
                       pvalues = totest[variants, "pval"], 
                       N       = 966, 
                       MAF     = totest[variants, "af"], 
                       type    = "quant")
    }
    
    return(dataset)
}

run_coloc_by_pop = function(locus, study1, study2, pop, totest1, totest2, manifest)
{
    if (nrow(manifest[ manifest$id == study1,]) > 0){trait_type1 = manifest[study1, "trait_type"]}
    if (nrow(manifest[ manifest$id == study2,]) > 0){trait_type2 = manifest[study2, "trait_type"]}
    
    if (nrow(manifest[ manifest$id == study1,]) == 0){trait_type1 = "qtl"}
    if (nrow(manifest[ manifest$id == study2,]) == 0){trait_type2 = "qtl"}
    
    if( trait_type1 %in% c("categorical", "icd10", "phecode")){variants1  = totest1[is.na(totest1[,paste("pval", pop, sep = "_")]) == FALSE & is.na(totest1[,paste("af_controls", pop, sep = "_")]) == FALSE, "id"]}
    if( trait_type1 %in% c("biomarkers", "continuous"       )){variants1  = totest1[is.na(totest1[,paste("pval", pop, sep = "_")]) == FALSE & is.na(totest1[,paste("af"         , pop, sep = "_")]) == FALSE, "id"]}
    if( trait_type1 %in% c("qtl"                            )){variants1  = totest1$id}
    
    
    if( trait_type2 %in% c("categorical", "icd10", "phecode")){variants2  = totest2[is.na(totest2[,paste("pval", pop, sep = "_")]) == FALSE & is.na(totest2[,paste("af_controls", pop, sep = "_")]) == FALSE, "id"]}
    if( trait_type2 %in% c("biomarkers", "continuous"       )){variants2  = totest2[is.na(totest2[,paste("pval", pop, sep = "_")]) == FALSE & is.na(totest2[,paste("af"         , pop, sep = "_")]) == FALSE, "id"]}
    if( trait_type2 %in% c("qtl"                            )){variants2  = totest2$id}
    
    if( trait_type1 %in% c("qtl")){pop1 = ""}
    if( trait_type2 %in% c("qtl")){pop2 = ""}
    
    if(!trait_type1 %in% c("qtl")){pop1 = pop}
    if(!trait_type2 %in% c("qtl")){pop2 = pop}
    
    variants2   = totest2$id
    variants    = intersect(variants1, variants2)
    
    if(length(variants) > 100)
    {
        dataset1        = create_dataset(study1, trait_type1, totest1, variants, pop1, manifest)
        dataset2        = create_dataset(study2, trait_type2, totest2, variants, pop2, manifest)
        coloc_mapped    = coloc.abf(dataset1 = dataset1, dataset2 = dataset2) 
        probs           = as.data.frame(t(coloc_mapped$summary))
        myres           = coloc_mapped$results
        myres           = myres[, c(which(colnames(myres) == "snp"), ncol(myres))]
        colnames(myres) = c("id", "pp_snp")
        myres           = myres[order(myres$pp_snp, decreasing = TRUE), ]
        myres$cum       = cumsum(myres$pp_snp)
        myres$cs        = FALSE
        to_cs           = myres[myres$cum >= 0.99, ][1, "id"]
        
        myres[myres$cum < 0.99 | myres$id == to_cs, "cs"] = TRUE
        
        out             = cbind(probs, myres[1, ])
    }else
    {
        out = data.frame(nsnps = 0, PP.H0.abf = 1, PP.H1.abf = 0, PP.H2.abf = 0, PP.H3.abf = 0, PP.H4.abf = 0, 
                         locus = locus, study1 = study1, study2 = study2, id = "", pp_snp = 0
                        )
        myres = data.frame(id = "", pp_snp = 0, cum = 1, cs = FALSE)
    }

    return(list(top = out, pp = myres))
}

combine_variants = function(totest_comb, coloc_tests, totest_list)
{
    myvars = sort(unique(unlist(lapply(totest_list, function(x){x$id}))))
    out    = matrix(0, nrow = length(myvars), ncol = nrow(totest_comb), dimnames = list(myvars, totest_comb$id))
    
    if(nrow(totest_comb) > 1)
    {
        for(id in totest_comb$id)
        {
            this = coloc_tests[[id]][["pp"]]
            
            if(nrow(this) > 1){out[ this$id, id] = this$pp_snp}
        }

        out = out[rownames(out) != "",]
    }else
    {
        this = coloc_tests[[totest_comb[1, "id"]]][["pp"]]
        
        if(nrow(this) > 1){out[ this$id, 1] = this$pp_snp}
    }
    
    return(out)
}

quiet = function(x) 
{ 
    sink(tempfile()) 
    on.exit(sink()) 
    invisible(force(x)) 
} 

analyze_locus = function(id, coloc_list, loci, manifest, to_return = FALSE)
{
    message(id)
    som            = coloc_list[[id]][["som"           ]]
    traits_cluster = coloc_list[[id]][["traits_cluster"]]
    eqtls          = coloc_list[[id]][["eqtls"         ]]
    traits         = coloc_list[[id]][["traits"        ]]
    coloc          = coloc_list[[id]][["coloc"         ]]   
    
    locus         = som[1, "locus"]
    coloc         = coloc[ coloc$PP.H4.abf >= 0.8,]
    coloc         = coloc[order(coloc$PP.H4.abf, decreasing = TRUE),]
    coloc$tr2type = paste(coloc$transcript_id, coloc$type)
    coord         = loci[locus,]
    coord         = paste0(coord$chrom, ":", coord$from, "-", coord$to)
    
    # get GWAS data
    totest_gwas_list        = lapply(traits_cluster, function(gwas){get_gwas_data(coord, manifest[gwas, "sumstat_file"])})
    names(totest_gwas_list) = traits_cluster
    
    # get QTL data
    qtls          = unique(coloc[,c("transcript_id", "gene_id", "phenotype", "type", "tr2type")])
    
    if(nrow(qtls) > 0)
    {
        qtls$qtl_file = paste("/frazer01/projects/CARDIPS/analysis/cardiac_qtls_combined/input/qtl/processing", qtls$phenotype, paste("qtl", qtls$transcript_id, "txt", sep = "."), sep = "/")

        qtl_list = lapply(1:nrow(qtls), function(ii)
        {
            indata = fread(qtls[ii, "qtl_file"], sep = "\t", header = TRUE, data.table = FALSE)
            type   =       qtls[ii, "type"    ]
            indata = indata[indata$type == type,]

            return(indata)
        })

        names(qtl_list) = qtls$tr2type
        
        totest_list = c(totest_gwas_list, qtl_list)
    }else
    {
        totest_list = totest_gwas_list
    }
    
    # Generate matrix for coloc (all vs. all)
    
    totest_comb           = as.data.frame(t(combn(names(totest_list), 2)))
    colnames(totest_comb) = c("study1", "study2")
    totest_comb$id        = paste(totest_comb$study1, totest_comb$study2, sep = ":")
    rownames(totest_comb) = totest_comb$id
    
    coloc_tests = lapply(totest_comb$id, function(id)
    {
        study1 = totest_comb[id, "study1"]
        study2 = totest_comb[id, "study2"]
        out    = run_coloc_by_pop(locus, study1, study2, "meta", totest_list[[study1]], totest_list[[study2]], manifest)
        
        return(out)
    })
    
    names(coloc_tests) = totest_comb$id
    
    pps                = combine_variants(totest_comb, coloc_tests, totest_list)
    var2pp             = data.frame(id = rownames(pps), pp_mean = as.numeric(rowMeans(pps)))
    var2pp             = var2pp[order(var2pp$pp_mean, decreasing = TRUE),]    
    out                = list(comb        = totest_comb, 
                              totest_list = totest_list, 
                              coloc_tests = coloc_tests,
                              pps         = pps        ,
                              var2pp      = var2pp
                             )
    
    saveRDS(out, paste0("pipeline/5.4.analyze_coloc_qtl_som_maps/coloc_data/", id, ".rds"))
    
    if(to_return == TRUE){return(out)}
}

id = "5_136864391_137972681.2" # atrial fibrillation + pulse rate
#id = "5_16423249_16671484.1" 

x = quiet(analyze_locus(id, coloc_list, loci, manifest, TRUE))

#invisible(lapply(totest, function(id){quiet(analyze_locus(id, coloc_list, loci, manifest, FALSE))}))


# Summarize colocalization results

In [ ]:
find_lead = function(id)
{
    x      = readRDS(paste0("pipeline/5.4.analyze_coloc_qtl_som_maps/coloc_data/", id, ".rds"))
    out    = x[["var2pp"]][1,]
    out    = cbind(locus2coloc[ locus2coloc$id == id, ], data.frame(variant_id = out$id, pp = out$pp_mean))
    
    if(out$eqtls_n >  0)
    {
        genes                = sort(unique(coordinates[ coordinates$transcript_id %in% unlist(strsplit(out$eqtls, "; ")), "gene_name"]))
        out$eqtls_gene_names = paste(genes, collapse = "; ")
        out$eqtls_genes_n    = length(genes)
    }else
    {
        out$eqtls_gene_names = ""
        out$eqtls_genes_n    = 0
    }
    
    out$traits_name = paste(sort(unique(manifest[ unlist(strsplit(out$traits, "; ")), "name"])), collapse = "; ")
    
    return(out)
}

id = "5_136864391_137972681.2" # atrial fibrillation + pulse rate
#find_lead(id)

pps = as.data.frame(rbindlist(lapply(totest, find_lead)), stringsAsFactors = FALSE)
pps = pps[order(pps$pp, decreasing = TRUE),]
fwrite(pps, "pipeline/5.4.analyze_coloc_qtl_som_maps/pps.txt", sep = "\t", col.names = TRUE, row.names = FALSE)

In [ ]:
nrow(pps[ pps$traits_n > 0 & pps$eqtls_n > 0 & pps$pp > 0.9, ])
nrow(pps[ pps$traits_n > 0 & pps$eqtls_n > 0 & pps$pp > 0.5 & pps$pp <= 0.9, ])


In [ ]:
pps[ pps$traits_n > 0 & pps$eqtls_n > 0 & pps$pp > 0.9, ]

In [ ]:
coordinates[ coordinates$transcript_id == "ENSG00000164867.11_5", ]